In [1]:
import pandas as pd

In [2]:
socios_tabla = pd.read_excel("../data/ICA_esqueleto.xlsx",sheet_name="R1.Principales países",skiprows=0)
socios_tabla.drop(1,inplace=True)
header = [socios_tabla.iloc[0].values, ["Exportación","Exportación","Exportación","Exportación",
                                        "Importación","Importación","Importación","Importación",
                                        "Saldo","Saldo"]]

header = ["<"+sup+" - "+inf+">" for inf, sup in zip(header[0],header[1])]
socios_tabla.columns = header

In [3]:
socios_tabla

,<Exportación - País>,<Exportación - Millones de dólares>,<Exportación - Variación porcentual>,<Exportación - Participación porcentual>,<Importación - País>,<Importación - Millones de dólares>,<Importación - Variación porcentual>,<Importación - Participación porcentual>,<Saldo - País>,<Saldo - Millones de dólares>
0,País,Millones de dólares,Variación porcentual,Participación porcentual,País,Millones de dólares,Variación porcentual,Participación porcentual,País,Millones de dólares
2,Octubre de 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Total,7901,15.1,100,Total,6074,15.8,100,Total,1827
4,China,1636,220.2,20.7,China,1501,13.5,24.7,Chile,294
5,Brasil,1123,0.8,14.2,Brasil,1247,24.5,20.5,Países Bajos,201
6,Estados Unidos,646,33.5,8.2,Estados Unidos,628,48.8,10.3,Perú,189
7,Chile,351,-16.4,4.4,Alemania,232,8.9,3.8,Viet Nam,160
8,Países Bajos,340,1.8,4.3,India,210,61.5,3.5,China,135
9,Viet Nam,241,2.6,3.1,Bolivia,150,105.5,2.5,Alemania,-132
10,Perú,213,3.4,2.7,Países Bajos,139,131.7,2.3,Brasil,-124


In [4]:
expo_socios = socios_tabla[16:].reset_index(drop=True)
expo_socios = expo_socios[expo_socios.columns[:4]] 
expo_socios.columns = ["País", "Exportación", "Variación porcentual","Participación porcentual"]
impo_socios = socios_tabla[16:].reset_index(drop=True)
impo_socios = impo_socios[impo_socios.columns[4:8]] 
impo_socios.columns = ["País", "Importación", "Variación porcentual","Participación porcentual"]
# impo_socios

In [5]:
CONTINENTES_DICT = {
    '1':'África',
    '2': "América",
    '3': "Asia",
    "4": "Europa",
    "5": "Oceanía",
    "9": "Indeterminado",
}
DIC_MESES = {
    1:"enero",
    2:"febrero",
    3:"marzo",
    4:"abril",
    5:"mayo",
    6:"junio",
    7:"julio",
    8:"agosto",
    9:"septiembre",
    10:"octubre",
    11: "noviembre",
    12: "diciembre"
}
def genera_tablas_con_continentes(df):
    df["continente"] = df.pais_cod.apply(lambda x: CONTINENTES_DICT[str(x)[0]])
    df.pais_descri = df.pais_descri.apply(lambda x: x.capitalize())
    df = df.sort_values(["anio","mes"],ascending=True).reset_index(drop=True)
    return df    

def genera_tablas_acumulado_y_mensual(df, comercio):
    valuacion = "cif" if comercio == "impo" else "fob"
    # Los datos mensuales
    ultimo_mes = df.mes.iloc[-1]
    ultimo_anio = df.anio.iloc[-1]
    socios_sas_mensual = df.copy()
    socios_sas_mensual["Variación porcentual"] = socios_sas_mensual.groupby(['pais_cod', 'pais_descri'])[valuacion].pct_change(12,fill_method=None)
    socios_sas_mensual = socios_sas_mensual[(socios_sas_mensual.anio == ultimo_anio)&(socios_sas_mensual.mes == ultimo_mes)].reset_index(drop=True)
    socios_sas_mensual["Participación Porcentual"] = socios_sas_mensual[valuacion] / socios_sas_mensual[valuacion].sum()
    
    # Acumulados
    socios_sas_acumulado = df[(df.mes <= ultimo_mes)].reset_index(drop=True).copy()
    socios_sas_acumulado = socios_sas_acumulado.groupby(["anio","pais_descri","pais_cod","continente"],as_index=False).sum().drop("mes",axis=1)
    socios_sas_acumulado["Variación porcentual"] = socios_sas_acumulado.groupby(['pais_cod', 'pais_descri'])[valuacion].pct_change(fill_method=None)
    socios_sas_acumulado = socios_sas_acumulado[socios_sas_acumulado.anio == 2022].reset_index(drop=True).drop("anio",axis=1)
    socios_sas_acumulado["Participación Porcentual"] = socios_sas_acumulado[valuacion] / socios_sas_acumulado[valuacion].sum()
    
    return socios_sas_acumulado, socios_sas_mensual


def genera_tabla_presentable_mensual(df_mensual, comercio):
    valuacion = "cif" if comercio == "impo" else "fob"
    socios_sas_mensual_presentable = df_mensual.rename({"pais_descri":"País",valuacion: "Millones de dólares"}, axis = 1).copy()
    socios_sas_mensual_presentable.drop(["anio","mes","pais_cod","continente"],axis = 1, inplace=True)
    socios_sas_mensual_presentable = socios_sas_mensual_presentable.sort_values("Millones de dólares",ascending=False).reset_index(drop=True)
    socios_sas_mensual_presentable = socios_sas_mensual_presentable[:10]
    total = df_mensual[valuacion].sum()
    sub_total = socios_sas_mensual_presentable[:10]["Millones de dólares"].sum()
    socios_sas_mensual_presentable.loc[-1] = ["Resto", (total-sub_total),0, (total-sub_total)/total]
    socios_sas_mensual_presentable = socios_sas_mensual_presentable.reset_index(drop=True)
    return socios_sas_mensual_presentable
    

def genera_tabla_presentable_acumulado(df_acumulado, comercio):
    #incorporar comercio
    valuacion = "cif" if comercio == "impo" else "fob"
    socios_sas_acumulado_presentable = df_acumulado.rename({"pais_descri":"País",valuacion: "Millones de dólares"}, axis = 1).copy()
    socios_sas_acumulado_presentable.drop(["pais_cod","continente"],axis = 1, inplace=True)
    socios_sas_acumulado_presentable = socios_sas_acumulado_presentable.sort_values("Millones de dólares",ascending=False).reset_index(drop=True)
    total = df_acumulado[valuacion].sum()
    sub_total = socios_sas_acumulado_presentable[:10]["Millones de dólares"].sum()
    socios_sas_acumulado_presentable = socios_sas_acumulado_presentable[:10]
    socios_sas_acumulado_presentable.loc[-1] = ["Resto", (total-sub_total),0, (total-sub_total)/total]
    socios_sas_acumulado_presentable = socios_sas_acumulado_presentable.reset_index(drop=True)
    return socios_sas_acumulado_presentable
    



In [6]:
#IMPORTACIONES
socios_sas_impo = pd.read_csv("../data/socios_impo.csv", sep=";")
socios_sas_impo.cif = socios_sas_impo.cif/1000000

#Agregamos continentes y capitalizamos los países
socios_sas_impo = genera_tablas_con_continentes(socios_sas_impo)

#última fecha de los datos. Útil para los gráficos
ultimo_mes = socios_sas_impo.mes.iloc[-1]
ultimo_anio = socios_sas_impo.anio.iloc[-1]

# Tablas mensuales y acumuladas. Importan las presentables
socios_sas_impo_acumulado, socios_sas_impo_mensual = genera_tablas_acumulado_y_mensual(socios_sas_impo, "impo")

socios_sas_impo_mensual_presentable = genera_tabla_presentable_mensual(socios_sas_impo_mensual, "impo")

socios_sas_impo_acumulado_presentable = genera_tabla_presentable_acumulado(socios_sas_impo_acumulado, "impo")
socios_sas_impo_acumulado_presentable

,País,Millones de dólares,Variación porcentual,Participación Porcentual
0,China,16320.054333,0.359871,0.213353
1,Brasil,14930.663958,0.328210,0.195190
2,Estados unidos,9770.275443,0.831315,0.127728
3,Alemania,2556.357358,0.091695,0.033419
4,Bolivia,2113.626527,1.204142,0.027632
5,Paraguay,1871.836050,-0.325183,0.024471
6,India,1766.536058,0.379027,0.023094
7,Tailandia,1703.667644,0.219497,0.022272
8,Mexico,1562.042938,0.235446,0.020421
9,Italia,1540.826619,0.283485,0.020143


In [7]:
#EXPORTACIONES
socios_sas_expo = pd.read_csv("../data/socios_expo.csv", sep=";")
socios_sas_expo.fob = socios_sas_expo.fob/1000000

#Agregamos continentes y capitalizamos los países
socios_sas_expo = genera_tablas_con_continentes(socios_sas_expo)

# Tablas mensuales y acumuladas. Importan las presentables
socios_sas_expo_acumulado, socios_sas_expo_mensual = genera_tablas_acumulado_y_mensual(socios_sas_expo, "expo")

socios_sas_expo_mensual_presentable = genera_tabla_presentable_mensual(socios_sas_expo_mensual, "expo")

socios_sas_expo_acumulado_presentable = genera_tabla_presentable_acumulado(socios_sas_expo_acumulado, "expo")

In [8]:
# Opción 1: DESCARTADA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#Separar más los títulos
#Hacer la opción para que sea gráfico único

specs = [[{'type':'domain'}, {'type':'domain'}]]
# cafe_colors =  ["#9e6e79","#798b92","#54a8ac","#2fc5c5","#09e1de","#43c1bd","#7ea19c","#b8817b","#d5716a","#f26059"]
fig = make_subplots(1, 2, specs=specs, subplot_titles=['Expo', 'Impo'])
fig.add_trace(go.Pie(labels=expo_socios.País, values= expo_socios.Exportación, scalegroup='one',), 1, 1)
fig.add_trace(go.Pie(labels=impo_socios.País, values= impo_socios.Importación, scalegroup='one',), 1, 2)

In [9]:
def plot_anillo_socios(df_presentable, acumulado:bool, ultimo_mes = ultimo_mes, ultimo_anio = ultimo_anio):
    fecha = f"Acumulado hasta {DIC_MESES[ultimo_mes]} de {ultimo_anio}" if acumulado else "Mes de noviembre"
    total = "{:,d}".format(int(round(df_presentable['Millones de dólares'].sum(),0))).replace(",",".")+"M"
    fig = go.Figure(data=[go.Pie(labels=df_presentable.País, values=df_presentable['Millones de dólares'], hole=.5)])
    fig.update_layout(title_text = f"Principales países de exportación:<br>{fecha}", template = None, font_family = "verdana",
                    margin = dict(t=70, l=10, r=10, b=30), separators = ",.",
                    annotations = [dict(text = f"{total}", showarrow = False, font_size = 20)],
                    showlegend= False,
                    paper_bgcolor='rgba(0,0,0,0)')
    fig.update_traces(textposition='inside', textinfo='label+percent+label')
    return fig

plot_anillo_socios(socios_sas_expo_mensual_presentable,  acumulado=False)
plot_anillo_socios(socios_sas_expo_acumulado_presentable,acumulado=True)

In [10]:
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    total_pa_color = df.cif.sum()
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'Importaciones'
        df_tree['value'] = dfg[value_column]
        
        df_tree['color'] = dfg[color_columns]/total_pa_color
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
        # df_all_trees = pd.concat([df_all_trees,df_tree] ,ignore_index=True)
    total = pd.Series(dict(id='Importaciones', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns]
                            ))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    #para reemplazar el nan del total
    df_all_trees.color.iloc[-1] = 1
    # df_all_trees =pd.concat([df_all_trees,total], ignore_index=True)
    return df_all_trees

def plot_sunburst_socios(df_impo, acumulado:bool):
    fecha = "acumulado hasta noviembre" if acumulado else "mes de noviembre"
    socios_sas_mensual_tree = build_hierarchical_dataframe(df=df_impo, levels=["pais_descri", "continente"],value_column="cif", color_columns = "cif")

    fig = go.Figure(go.Sunburst(
        labels=socios_sas_mensual_tree['id'],
        parents=socios_sas_mensual_tree['parent'],
        values=socios_sas_mensual_tree['value'],
        branchvalues='total',
        marker=dict(
            colors=socios_sas_mensual_tree['color'],
            colorscale='Sunsetdark',
            # cmid=average_score
            ),
        hovertemplate='<b>%{label} </b> <br> CIF: $%{value}<br> Proporcion: %{color:.2%}', #Me jode la proporcion, sí o sí entra por color
        name='',
        maxdepth=2
        ))
    fig.update_layout(title_text = f"Principales socios comerciales {fecha}: <br>Importaciones", template = None,
                    margin = {"t":70, "b":20, "l":10, "r":10},separators = ",." ,
                    #   uniformtext=dict(minsize=10, mode='hide'),
                    font_family = "verdana",
                    paper_bgcolor='rgba(0,0,0,0)',
                    ) 
    
    return fig

plot_sunburst_socios(socios_sas_impo_mensual, acumulado=False)
plot_sunburst_socios(socios_sas_impo_acumulado, acumulado=True)

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14548\1352287898.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14548\1352287898.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14548\1352287898.py:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14548\1352287898.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14548\1352287898.py:22: Fu